In [1]:
#0.導入套件(以前程式抓來的，可能有些用不上)
import numpy as np
import pandas as pd
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler      #torch的太麻煩了，用sklearn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [2]:
#1.讀取檔案，設定參數
#----------------------------------------
data = pd.read_csv("data.csv")
#data = data.rename(columns={"x'": "xp"})     原本考量x'往後需要變成x\'，但發現後面只出現一次，而且用不同之引號能解決'的問題，故注解掉
X = data[['a','m','V0','n',"x'"]].values           
y = data['psi_raw'].values.reshape(-1, 1)

#2.Normalization(標準化)
#---------------------------------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# train80% , test20%
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Tensor 化
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test  = torch.tensor(X_test,  dtype=torch.float32).to(device)
y_test  = torch.tensor(y_test,  dtype=torch.float32).to(device)

In [3]:
#3.模型
#---------------------------------------------------------
class ANN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(5 , 128),
            nn.Tanh(),
            nn.Linear(128 , 64),
            nn.Tanh(),
            nn.Linear(64 , 32),
            nn.Tanh(),
            nn.Linear(32 , 1),
        )
    def forward(self , x):
        return self.net(x)
model = ANN().to(device)

optimizer = torch.optim.Adam(model.parameters() , lr=1e-3)
loss_fn = nn.MSELoss()

In [4]:
# 4.訓練
#---------------------------------------------------
epochs = 20000
for i in range(epochs):
    optimizer.zero_grad()
    pred = model(X_train)
    loss = loss_fn(pred , y_train)
    loss.backward()
    optimizer.step()
    if(i == 0 or (i + 1) % 500 == 0):             #每500代輸出一次，順便確認首代loss
        with torch.no_grad():
            test_pred = model(X_test)
            test_loss = loss_fn(test_pred, y_test)                                
        print(f"Epoch {i}, Train = {loss.item():.5f}, Test = {test_loss.item():.5f}") 

Epoch 0, Train = 0.12323, Test = 0.10971
Epoch 499, Train = 0.09707, Test = 0.09745
Epoch 999, Train = 0.07493, Test = 0.07574
Epoch 1499, Train = 0.03950, Test = 0.04011
Epoch 1999, Train = 0.02606, Test = 0.02629
Epoch 2499, Train = 0.01991, Test = 0.02017
Epoch 2999, Train = 0.01713, Test = 0.01735
Epoch 3499, Train = 0.01465, Test = 0.01488
Epoch 3999, Train = 0.01211, Test = 0.01231
Epoch 4499, Train = 0.01001, Test = 0.01020
Epoch 4999, Train = 0.00913, Test = 0.00935
Epoch 5499, Train = 0.00851, Test = 0.00875
Epoch 5999, Train = 0.00793, Test = 0.00818
Epoch 6499, Train = 0.00739, Test = 0.00768
Epoch 6999, Train = 0.00682, Test = 0.00715
Epoch 7499, Train = 0.00623, Test = 0.00660
Epoch 7999, Train = 0.00573, Test = 0.00614
Epoch 8499, Train = 0.00535, Test = 0.00579
Epoch 8999, Train = 0.00503, Test = 0.00549
Epoch 9499, Train = 0.00474, Test = 0.00518
Epoch 9999, Train = 0.00450, Test = 0.00493
Epoch 10499, Train = 0.00430, Test = 0.00471
Epoch 10999, Train = 0.00412, Test =

In [5]:
#5.儲存
#--------------------------------------------------
torch.save(model.state_dict(), "ann_model.pth")

# 儲存 scaler
import joblib
joblib.dump(scaler, "ann_scaler.pkl")

print("Model and scaler saved.")

Model and scaler saved.
